# Makefile Tutorial

- Makefile is a way of automating software building procedure and other complex tasks with dependencies.
- The make utility is a software tool for managing and maintaining
computer programs consisting many component files. The make utility
automatically determines which pieces of a large program need to be
recompiled, and issues commands to recompile them. 
- Makefile contains: dependency rules, macros and suffix(or implicit)
rules. 

## Install

Install make
```
sudo apt install make
```

In [ ]:
!sudo apt install build-essential

In [2]:
# check version

!make -version

GNU Make 4.3
Built for x86_64-pc-linux-gnu
Copyright (C) 1988-2020 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [1]:
# check build-essential
!ls /usr/bin/make

/usr/bin/make


## Overview

A simple makefile consists of "rules" with the following shape:

```
target ... : dependencies ...
        command
        ...
        ...
```

- A `target` is usually the name of a file that is generated by a program; examples of targets are executable or object files. A target can also be the name of an action to carry out, such as `clean'
- A `dependency` is a file that is used as input to create the target. A target often depends on several files.
- A `command` is an action that make carries out. A rule may have more than one command, each on its own line. Please note: you need to put a tab character at the beginning of every command line!
- A `rule` explains how and when to remake certain files which are the targets of the particular rule. make carries out the commands on the dependencies to create or update the target.
- We split each long line into two lines using backslash-newline
- To use this makefile to create the executable file called `edit', type:

### A Simple Makefile

### 1. echo

In [45]:
%%writefile Makefile

hello:
	echo "Hello, World 1"
	echo "This line will always print, \
		because the file hello does not exist."

Overwriting Makefile


In [46]:
!make

echo "Hello, World 1"
Hello, World 1
echo "This line will always print, \
	because the file hello does not exist."
This line will always print, 	because the file hello does not exist.


- We have one target called hello
- This target has two commands
- This target has no prerequisites

### 2. Run c file

In [54]:
%%writefile blah.c
int main() { return 0; }

Writing blah.c


In [55]:
%%writefile Makefile
blah:
	cc blah.c -o blah

Overwriting Makefile


In [56]:
!make

make: 'blah' is up to date.


- The first time you run this, blah will be created. The second time, you'll see make: 'blah' is up to date. That's because the blah file already exists. 

- But there's a problem: if we modify blah.c and then run make, nothing gets recompiled.

In [57]:
%%writefile Makefile
blah: blah.c
	cc blah.c -o blah

Overwriting Makefile


In [58]:
!make

cc blah.c -o blah


- The first target is selected, because the first target is the default target
- This has a prerequisite of blah.c
- Make decides if it should run the blah target. It will only run if blah doesn't exist, or blah.c is newer than blah

### 3. Simple piepline

In [59]:
%%writefile Makefile
blah: blah.o
	cc blah.o -o blah # Runs third

blah.o: blah.c
	cc -c blah.c -o blah.o # Runs second

# Typically blah.c would already exist, but I want to limit any additional required files
blah.c:
	echo "int main() { return 0; }" > blah.c # Runs first

Overwriting Makefile


In [60]:
!make

cc -c blah.c -o blah.o # Runs second
cc blah.o -o blah # Runs third


- Make selects the target blah, because the first target is the default target
- blah requires blah.o, so make searches for the blah.o target
- blah.o requires blah.c, so make searches for the blah.c target
- blah.c has no dependencies, so the echo command is run
- The cc -c command is then run, because all of the blah.o dependencies are finished
- The top cc command is run, because all the blah dependencies are finished

In [62]:
%%writefile Makefile

some_file: other_file
	echo "This will always run, and runs second"
	touch some_file

other_file:
	echo "This will always run, and runs first"

Overwriting Makefile


In [63]:
!make

echo "This will always run, and runs first"
This will always run, and runs first
echo "This will always run, and runs second"
This will always run, and runs second
touch some_file


- It will always run both targets, because some_file depends on other_file, which is never created.

### 4. Make clean

`clean` is often used as a target that removes the output of other targets, but it is not a special word in Make. You can run `make` and `make clean` on this to create and delete some_file.

clean is doing two new things here:
- It's a target that is not first (the default), and not a prerequisite. That means it'll never run unless you explicitly call make clean
- It's not intended to be a filename. If you happen to have a file named clean, this target won't run, which is not what we want. See `.PHONY` later in this tutorial on how to fix this

In [73]:
%%writefile Makefile
some_file: 
	touch some_file

clean:
	rm -f some_file

Overwriting Makefile


In [70]:
!make

touch some_file


In [74]:
!make clean

rm -f some_file


### 5. Variable

Variables can only be strings. You'll typically want to use `:=`, but `=` also works. 

In [75]:
%%writefile Makefile

files := file1 file2
some_file: $(files)
	echo "Look at this variable: " $(files)
	touch some_file

file1:
	touch file1
file2:
	touch file2

clean:
	rm -f file1 file2 some_file

Overwriting Makefile


In [76]:
!make

touch file1
touch file2
echo "Look at this variable: " file1 file2
Look at this variable:  file1 file2
touch some_file


In [78]:
!make clean

rm -f file1 file2 some_file


### 6. Reference

Reference variables using either `${}` or `$()`

In [81]:
%%writefile Makefile

x := dude

all:
	echo $(x)
	echo ${x}

	# Bad practice, but works
	echo $x 

Overwriting Makefile


In [82]:
!make

echo dude
dude
echo dude
dude
# Bad practice, but works
echo dude 
dude


### 7. String Variable

Single or double quotes have no meaning to Make. They are simply characters that are assigned to the variable. Quotes are useful to shell/bash, though, and you need them in commands like `printf`.

In [79]:
%%writefile Makefile

a := one two # a is assigned to the string "one two"
b := 'one two' # Not recommended. b is assigned to the string "'one two'"
all:
	printf '$a'
	printf $b

Overwriting Makefile


In [80]:
!make

printf 'one two '
one two printf 'one two' 
one two

### 8. The all target

Making multiple targets and you want all of them to run? Make an `all` target. Since this is the first rule listed, it will run by default if make is called without specifying a target.